<a href="https://colab.research.google.com/github/ovokojo/fintequity/blob/main/SEC_API_Tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install SEC API & load API Key from file (Get a free API key at https://sec-api.io/)
!pip install python-dotenv
!pip install -q sec-api

from dotenv import dotenv_values
from google.colab import files

# Upload the .env file from your local machine
uploaded = files.upload()

# Get the file name of the uploaded file
filename = next(iter(uploaded))

# Load the environment variables from the uploaded file
env_vars = dotenv_values(filename)

# Extract the API key from the environment variables
API_KEY = env_vars.get("API_KEY")

print("API key loaded successfully")

Saving .ENV to .ENV
API key loaded successfully


In [18]:
# Query API - Search and filter all 18+ million filings and exhibits published on SEC EDGAR since 1993

import requests

def getLatest10K(ticker: str):
    """
    Retrieves the most recent Form 10-K filing for a given stock ticker symbol.

    Args:
    - ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.

    Returns:
    - dict: A dictionary containing the JSON response with the latest Form 10-K filing
            information if the request is successful (HTTP status code 200).
    - int: The HTTP status code of the response if the request is unsuccessful.
    """

    endpoint = f"https://api.sec-api.io?token={API_KEY}"
    params = {
        "query": {
          "query_string": {
            "query":  f"ticker:{ticker} AND formType:\"10-K\""
          }
        },
        "from": "0",
        "size": "1",
        "sort": [{ "filedAt": { "order": "desc" }}]
    }
    response = requests.post(endpoint, json=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        return response.status_code

# Example
# latest10K = getLatest10K('TSLA')

In [24]:
# Extractor API - Extract any text section from 10-Q, 10-K, and 8-K SEC filings.

import requests

def getLatest10kSection(ticker: str, section: str):
    """
    Retrieves a specified section from the latest 10-K filing of any stock using the SEC Query API & the extractor API

    Args:
        ticker (str): The stock ticker symbol of the company for which the Form 10-K is requested.
        section (str): Specific section of the filing to retrieve (e.g., "1A" for the Risk Factors section).
        10-K supported section codes: 1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15

    Returns:
        text or int: The extracted content in JSON format if request is successful; otherwise, the HTTP status code.
    """

    latest10K = getLatest10K(ticker)
    url_10k = latest10K['filings'][0]['linkToFilingDetails']

    endpoint = "https://api.sec-api.io/extractor"

    params = {
        "token": API_KEY,
        "url": url_10k,
        "item": section,
        "type": "text"
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        print(response.text)
        return response.text
    else:
        print(f"Request failed with code: {response.status_code}")
        return response.status_code

# Example
# getLatest10kSection("WDAY", "1A")


In [ ]:
# Next steps: analyze & summarize section with AI